In [1]:
import os
from langchain.document_loaders import CSVLoader, DirectoryLoader, JSONLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter


os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
directory = "../data/google_drive_sahaj/All/Projects/ClearChannel"
docs = []
print("Loading json documents...")
docs = DirectoryLoader(path=directory, glob="**/*.json", recursive=True, show_progress=True, silent_errors=True, loader_cls=JSONLoader, loader_kwargs={"jq_schema": ".", "text_content": False}).load()
print("Loading csv documents...")
docs += DirectoryLoader(path=directory, glob="**/*.csv", recursive=True, show_progress=True, silent_errors=True, loader_cls=CSVLoader, loader_kwargs={"csv_args":{"delimiter": "\n"}}).load()
print("Loading md documents...")
docs += DirectoryLoader(path=directory, glob="**/*.md", recursive=True, show_progress=True, silent_errors=True, loader_kwargs={"autodetect_encoding": True, "mode": "elements"}).load()
print("Loading all remaining non-json/csv/md documents...")
# load all non-json/csv/md files as text
docs += DirectoryLoader(path=directory, glob="**/*[!(.json|.csv|.md)]", recursive=True, show_progress=True, silent_errors=True, loader_kwargs={"autodetect_encoding": True, "mode": "elements"}).load()


Loading json documents...


0it [00:00, ?it/s]


Loading csv documents...


0it [00:00, ?it/s]


Loading md documents...


0it [00:00, ?it/s]


Loading all remaining non-json/csv/md documents...


  0%|                                                                                                                                                                                                                                                  | 0/14 [00:00<?, ?it/s]Error loading file ../data/google_drive_sahaj/All/Projects/ClearChannel/Programmatic Vision Jan 2023/CCI_AlignmentWorkshop_Programmatic_15Feb23.pptx: unstructured package not found, please install it with `pip install unstructured`
Error loading file ../data/google_drive_sahaj/All/Projects/ClearChannel/Discovery July 2022/CCI LAUNCHPAD Discovery Outcomes.docx: unstructured package not found, please install it with `pip install unstructured`
Error loading file ../data/google_drive_sahaj/All/Projects/ClearChannel/Discovery July 2022/CCI LAUNCHPAD Discovery Outcomes.pptx: unstructured package not found, please install it with `pip install unstructured`
Error loading file ../data/google_drive_sahaj/All/Projects/ClearChannel/